📊 **EPL Visualisations Notebook**
- This notebook provides an exploratory data analysis (EDA) of English Premier League (EPL) match data. It focuses on key aspects of team and match performance using visual insights.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt